In [13]:
date = '2019-09-01'

In [14]:
import pandas as pd
pd.__version__

'0.24.2'

In [22]:
y, m, d = date.split('-')
input_base = 's3://edill-data/cleaned-logs'
#output_base = 's3://edill-data/conda/hourly'
input_file = f'{input_base}/{y}/{m}/{date}.parquet'
#output_file = f'{output_base}/{y}/{m}/{date}.parquet'
input_file

's3://edill-data/cleaned-logs/2016/10/2016-10-28.parquet'

In [23]:
df = pd.read_parquet(input_file)
df.dtypes

time            datetime64[ns]
data_source           category
pkg_name              category
pkg_version           category
pkg_platform          category
pkg_python            category
bytes                    int64
day_name                object
hour                     int64
counts                   int64
dtype: object

In [24]:
df.head().T

,0,1,2,3,4
time,2016-10-28 17:00:00,2016-10-28 17:00:00,2016-10-28 17:00:00,2016-10-28 17:00:00,2016-10-28 17:00:00
data_source,anaconda,anaconda,anaconda,anaconda,anaconda
pkg_name,anaconda-client,appscript,appscript,bzip2,bzip2
pkg_version,1.5.3,1.0.1,1.0.1,1.0.6,1.0.6
pkg_platform,linux-64,osx-64,osx-64,linux-64,linux-64
pkg_python,3.5,2.7,2.7,,
bytes,119691,125038,125039,85291,85292
day_name,Friday,Friday,Friday,Friday,Friday
hour,17,17,17,17,17
counts,1,3,2,4,1


In [25]:
import cudf

In [26]:
cdf = cudf.read_parquet(input_file)
cdf.dtypes

time            datetime64[ms]
data_source             object
pkg_name                object
pkg_version             object
pkg_platform            object
pkg_python              object
bytes                    int64
day_name                object
hour                     int64
counts                   int64
dtype: object

In [27]:
cdf.head().to_pandas().T

,0,1,2,3,4
time,2016-10-28 17:00:00,2016-10-28 17:00:00,2016-10-28 17:00:00,2016-10-28 17:00:00,2016-10-28 17:00:00
data_source,anaconda,anaconda,anaconda,anaconda,anaconda
pkg_name,anaconda-client,appscript,appscript,bzip2,bzip2
pkg_version,1.5.3,1.0.1,1.0.1,1.0.6,1.0.6
pkg_platform,linux-64,osx-64,osx-64,linux-64,linux-64
pkg_python,3.5,2.7,2.7,,
bytes,119691,125038,125039,85291,85292
day_name,Friday,Friday,Friday,Friday,Friday
hour,17,17,17,17,17
counts,1,3,2,4,1


In [4]:
pd.read_parquet(input_file, engine='fastparquet').to_parquet(output_file, engine='pyarrow')

In [6]:
for col in ['data_source', 'pkg_name', 'pkg_version','pkg_platform','pkg_python']:
    df[col]=df[col].astype('category')


In [7]:
df.dtypes

time            datetime64[ns]
data_source           category
pkg_name              category
pkg_version           category
pkg_platform          category
pkg_python            category
counts                   int64
dtype: object

In [37]:
import cudf

In [57]:
engine = 'pyarrow'
written_by_pyarrow = f'{date}-{engine}.parquet.SNAPPY'
df.to_parquet(written_by_pyarrow, compression="SNAPPY", engine=engine)
written_by_pyarrow

'2019-01-01-pyarrow.parquet.SNAPPY'

In [58]:
engine = 'fastparquet'
written_by_fastparquet = f'{date}-{engine}.parquet.SNAPPY'
df.to_parquet(written_by_fastparquet, compression="SNAPPY", engine=engine)
written_by_fastparquet

'2019-01-01-fastparquet.parquet.SNAPPY'

In [68]:
engine = 'cudf'
written_by_cudf = f'{date}-{engine}.parquet.SNAPPY'
cudf.from_pandas(df).to_parquet(written_by_cudf, compression="SNAPPY")
written_by_cudf

'2019-01-01-cudf.parquet.SNAPPY'

In [51]:
import itertools

In [ ]:
funcs = [pd.read_parquet, cudf.read_parquet]
engines = ['pyarrow', 'cudf', 'fastparquet']
for func, engine in itertools.product()

In [43]:
pd.read_parquet(written_by_pyarrow, engine='pyarrow').dtypes

time            datetime64[ns]
data_source             object
pkg_name                object
pkg_version             object
pkg_platform            object
pkg_python              object
counts                   int64
dtype: object

In [45]:
pd.read_parquet(written_by_pyarrow, engine='fastparquet').dtypes

time            datetime64[ns]
data_source           category
pkg_name              category
pkg_version           category
pkg_platform          category
pkg_python            category
counts                   int64
dtype: object

In [46]:
pd.read_parquet(written_by_fastparquet, engine='pyarrow').dtypes

TypeError: expected bytes, dict found

In [48]:
pd.read_parquet(written_by_fastparquet, engine='fastparquet').dtypes

time            datetime64[ns]
data_source           category
pkg_name              category
pkg_version           category
pkg_platform          category
pkg_python            category
counts                   int64
dtype: object